In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
from scipy.stats import mode

temperature = pd.read_csv("기온.csv", encoding="cp949")
rainfall = pd.read_csv("강수량.csv", encoding="cp949")
wind = pd.read_csv("바람.csv", encoding="cp949")
humidity = pd.read_csv("습도.csv", encoding="cp949")
sunshine = pd.read_csv("일조일사.csv", encoding="cp949")

temperature = temperature.drop([temperature.columns[0], temperature.columns[1]], axis=1)
rainfall = rainfall.drop([rainfall.columns[0], rainfall.columns[1]], axis=1)
wind = wind.drop([wind.columns[0], wind.columns[1]], axis=1)
humidity = humidity.drop([humidity.columns[0], humidity.columns[1]], axis=1)
sunshine = sunshine.drop([sunshine.columns[0], sunshine.columns[1]], axis=1)

temperature.rename(columns = {"\t최고기온시각": "최고기온시각"}, inplace=True)
wind.rename(columns = {"\t최대풍속시각":"최대풍속시각", "\t최대순간풍속시각": "최대순간풍속시각"}, inplace=True)

merge_data = pd.merge(temperature, rainfall, on='일시')
merge_data = pd.merge(merge_data, wind, on='일시')
merge_data = pd.merge(merge_data, humidity, on='일시')
merge_data = pd.merge(merge_data, sunshine, on='일시')
merge_data = merge_data[:2158]

merge_data.to_csv("observed.csv", mode='w', encoding='cp949')
observed = pd.read_csv("observed.csv", encoding='cp949')
observed = observed.drop([observed.columns[0]], axis=1) # 저장하면서 index열이 생김. 이를 지우기 위함.

weather = pd.read_csv("기상(타겟).csv", encoding="cp949", low_memory=False) # low_memory = 경고 무시용

weather = weather[weather['지역']=='전라북도']
weather = weather.drop(weather.columns[0], axis=1)

weather = weather.sort_values(by=['예보시각'])
weather = weather.reset_index(drop=True)

forecast_score = {'맑음':0,
                  '구름조금':1, '구름많음':1.1,
                  '흐림':2,
                  '구름많고 소나기':3, '구름많고 비':3.1, '구름많고 비/눈':3.2, '구름많고 눈/비':3.2, '구름많고 눈':3.3,
                  '흐리고 비':4, '흐리고 눈/비':4.1, '흐리고 비/눈':4.1, '흐리고 눈':4.2}
re_score = {v:k for k,v in forecast_score.items()}
from scipy.stats import mode

def select_forecast(date):
    temp = 0
    for forecast in date:
        if forecast_score[forecast] > temp:
            temp = forecast_score[forecast]
    return re_score[temp]
weather_copy = weather.copy()
for i, w in enumerate(weather_copy['예보시각']):
    weather_copy['예보시각'][i] = w[:10]
    
weather_copy = weather_copy.drop_duplicates(['예보시각'])
weather_copy = weather_copy.reset_index(drop=True)
weather_copy['예보'] = ''

w=0
start = 0
for i in range(len(weather)):
    if weather_copy["예보시각"][w] not in weather["예보시각"][i]:
        weather_copy["예보"][w] = select_forecast(weather["예보"][start:i])
        w += 1
        start = i
        
weather = weather_copy

weather = weather.drop([weather.columns[0]], axis=1)

weather.to_csv("forecast.csv", mode='w', encoding='cp949')
forecast = pd.read_csv("forecast.csv", encoding='cp949')
forecast = forecast.drop([forecast.columns[0]], axis=1) # 저장하면서 index열이 생김. 이를 지우기 위함.

forecast.rename(columns = {"예보시각": "일시"}, inplace=True)
df = pd.merge(observed, forecast, on='일시')
df.to_csv("df.csv", encoding='cp949')
df = pd.read_csv("df.csv", encoding='cp949')
df = df.drop([df.columns[0]], axis=1) # 저장하면서 index열이 생김. 이를 지우기 위함.

df


,일시,평균기온(℃),최고기온(℃),최고기온시각,최저기온(℃),최저기온시각,일교차,강수량(mm),1시간최다강수량(mm),1시간최다강수량시각,...,최대풍속시각,최대순간풍속(m/s),최대순간풍속풍향(deg),최대순간풍속시각,평균습도(%rh),최저습도(%rh),일조합,일조율,일사합,예보
0,2016-01-01,2.8,7.7,14:24,-2.0,6:59,9.7,NaN,NaN,NaN,...,3:04,4.2,290.0,16:29,74.4,52.0,4.3,43.9,7.90,구름조금
1,2016-01-02,7.6,12.9,14:59,2.7,7:01,10.2,NaN,NaN,NaN,...,13:40,7.7,230.0,13:37,73.3,59.0,5.6,57.1,8.23,구름많음
2,2016-01-03,8.4,14.0,14:17,5.4,23:19,8.6,NaN,NaN,NaN,...,14:20,5.4,270.0,15:44,81.1,49.0,7.3,74.5,9.88,구름많음
3,2016-01-04,5.9,9.6,14:01,1.6,23:53,8.0,NaN,NaN,NaN,...,12:12,6.6,290.0,13:40,70.1,42.0,7.1,72.4,9.26,구름많음
4,2016-01-05,0.7,4.1,15:27,-1.9,23:25,6.0,NaN,NaN,NaN,...,15:40,6.7,320.0,15:48,53.0,28.0,3.7,37.8,6.47,구름많음
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2153,2021-11-23,2.8,5.4,15:10,1.0,8:07,4.4,2.6,NaN,NaN,...,5:02,5.5,270.0,16:37,81.5,60.0,2.1,20.8,5.94,흐리고 눈
2154,2021-11-24,6.8,11.5,13:53,2.1,7:07,9.4,1.0,NaN,NaN,...,14:35,8.8,250.0,14:21,72.5,46.0,7.5,74.3,9.06,흐리고 눈
2155,2021-11-25,9.6,14.5,13:18,5.9,23:59,8.6,NaN,NaN,NaN,...,14:04,8.1,270.0,16:14,63.6,34.0,6.7,67.0,9.07,흐림
2156,2021-11-26,5.9,11.0,14:12,1.5,7:28,9.5,NaN,NaN,NaN,...,17:12,5.2,270.0,16:59,63.9,28.0,9.1,91.0,11.86,구름많음
